## Manipulating fake sales data from a bookstore

##### In this repository we will do a slightly more complex exercise. We will create a fake datalake from a bookstore. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("Spark_Dataframe")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/05/02 20:34:16 WARN Utils: Your hostname, DESKTOP-VMHJUON resolves to a loopback address: 127.0.1.1; using 192.168.92.85 instead (on interface eth0)
24/05/02 20:34:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/02 20:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#create a clientes dataframe
clientes_df = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/clientes.csv')
)

In [4]:
#create a vendedores dataframe
vendedores_df = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/vendedores.csv')
)

In [5]:
#create a transacoes dataframe
transacoes_df = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/transacoes.csv')
)

In [6]:
#create a livros dataframe
livros_df = (
    spark
    .read
    .option('delimiter', ',')
    .option('header', True)
    .option('inferSchema', True)
    .option('encoding', 'ISO-8859-1')
    .csv('lake/livros.csv')
)

In [7]:
#create a view to use in sql queries
clientes_df.createOrReplaceTempView('cliente')
vendedores_df.createOrReplaceTempView('vendedor')
transacoes_df.createOrReplaceTempView('transacoes')
livros_df.createOrReplaceTempView('livro')



In [8]:
# show quantity of rows in each table 
spark.sql('''
          SELECT count(*) as Quantidade, "cliente" as tabela from cliente c
            union
            SELECT count(*) as Quantidade, "vendedor" as tabela from vendedor v 
            union
            SELECT count(*) as Quantidade, "Transacoes" as tabela from transacoes t
            union
            SELECT count(*) as Quantidade, "livro" as tabela from livro l;  
          ''').show()

+----------+----------+
|Quantidade|    tabela|
+----------+----------+
|       220|   cliente|
|        10|  vendedor|
|       714|Transacoes|
|       120|     livro|
+----------+----------+



#
